In [1]:
from datetime import datetime

import dask
import dask.dataframe as dd
import pandas as pd

from dask.distributed import Client

In [2]:
with dask.config.set({'temporary_directory': '~/loqutus/data/tmp'}):
    pass
client = Client()

In [3]:
filepath = "../data/yellow_tripdata_2019-01.csv"
datetime_format = "%Y-%m-%d %H:%M:%S"
QUERY_MONTH = "2019-1"

In [4]:
df = dd.read_csv(filepath, blocksize="10MB")

In [5]:
columns_of_interest = [
    "VendorID", 
    "PULocationID", 
    "trip_distance",
    "fare_amount",
    "payment_type",
    "tpep_pickup_datetime"
]
df = df[columns_of_interest].rename(columns={"tpep_pickup_datetime":"pickup_datetime"})
#df = df.head(1000)
type(df)

dask.dataframe.core.DataFrame

In [6]:
df["pickup_datetime"] = df.apply(
    lambda rec: datetime.strptime(rec["pickup_datetime"], datetime_format), 
    #lambda rec: pd.to_datetime(rec["pickup_datetime"]),
    axis=1,
    meta = ('pickup_datetime', 'datetime64[ns]')
)

In [81]:
df.head()

,VendorID,PULocationID,trip_distance,fare_amount,payment_type,pickup_datetime
0,1,151,1.5,7.0,1,2019-01-01 00:46:40
1,1,239,2.6,14.0,1,2019-01-01 00:59:47
2,2,236,0.0,4.5,1,2018-12-21 13:48:30
3,2,193,0.0,3.5,2,2018-11-28 15:52:25
4,2,193,0.0,52.0,2,2018-11-28 15:56:57


In [7]:
df["year_month"] = df.apply(
    lambda rec: f'{rec["pickup_datetime"].year}-{rec["pickup_datetime"].month}', 
    axis=1,
    meta = ("string")
)

In [8]:
df = df.set_index("year_month")

In [9]:
pd_df = df[df.index==QUERY_MONTH].compute()

NotImplementedError: dask.array<eq, shape=(nan,), dtype=bool, chunksize=(nan,), chunktype=numpy.ndarray>

In [10]:
df.head()

,VendorID,PULocationID,trip_distance,fare_amount,payment_type,pickup_datetime
year_month,,,,,,
